In [ ]:
%pip install nltk

import nltk
import nltk.tokenize
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
import spacy
import os


In [ ]:
# print(spacy.__version__)
!py -m spacy download ru_core_news_sm

In [ ]:
nlp = spacy.load('ru_core_news_sm')

In [35]:
stop = stopwords.words("russian")

def text_preproc(adress):
    text = open(adress, "r").read()
    text = WhitespaceTokenizer().tokenize(text)
    words_filtered = [word for word in text if word not in stop]
    
    return words_filtered


In [41]:
tanka = text_preproc("6_texts\Бунин Иван. Танька - royallib.ru.txt")
print (tanka)
antons_apples = text_preproc("6_texts\\antonovskie-yabloki.txt")
antons_apples

['Таньке', 'стало', 'холодно,', 'проснулась.', 'Высвободив', 'руки', 'попонки,', 'которую', 'неловко', 'закуталась', 'ночью,', 'Танька', 'вытянулась,', 'глубоко', 'вздохнула', 'сжалась.', 'Но', 'все-таки', 'холодно.', 'Она', 'подкатилась', 'самую', '«голову»', 'печи', 'прижала', 'Ваську.', 'Тот', 'открыл', 'глаза', 'взглянул', 'светло,', 'смотрят', 'сна', 'здоровые', 'дети.', 'Потом', 'повернулся', 'бок', 'затих.', 'Танька', 'стала', 'задремывать.', 'Но', 'избе', 'стукнула', 'дверь:', 'мать,', 'шурша,', 'протаскивала', 'сенец', 'охапку', 'соломы.', '–', 'Холодно,', 'тетка?', '–', 'спросил', 'странник,', 'лежа', 'конике.', '–', 'Нет,', '–', 'ответила', 'Марья,', '–', 'туман.', 'А', 'собаки', 'валяются,', '–', 'беспременно', 'метели.', 'Она', 'искала', 'спичек', 'гремела', 'ухватами.', 'Странник', 'спустил', 'ноги', 'коника,', 'зевал', 'обувался.', 'В', 'окна', 'брезжил', 'синеватый', 'холодный', 'свет', 'утра;', 'лавкой', 'шипел', 'крякал', 'проснувшийся', 'хромой', 'селезень.', 'Телено

['Иван',
 'Алексеевич',
 'Бунин',
 'Антоновские',
 'яблоки',
 'I',
 '...Вспоминается',
 'ранняя',
 'погожая',
 'осень.',
 'Август',
 'теплыми',
 'дождиками,',
 'нарочно',
 'выпадавшими',
 'сева,',
 'дождиками',
 'самую',
 'пору,',
 'середине',
 'месяца,',
 'около',
 'праздника',
 'св.',
 'Лаврентия.',
 'А',
 '"осень',
 'зима',
 'хороши',
 'живут,',
 'коли',
 'Лаврентия',
 'вода',
 'тиха',
 'дождик".',
 'Потом',
 'бабьим',
 'летом',
 'паутины',
 'село',
 'поля.',
 'Это',
 'добрый',
 'знак:',
 '"Много',
 'тенетника',
 'бабье',
 'лето',
 '--',
 'осень',
 'ядреная"...',
 'Помню',
 'раннее,',
 'свежее,',
 'тихое',
 'утро...',
 'Помню',
 'большой,',
 'весь',
 'золотой,',
 'подсохший',
 'поредевший',
 'сад,',
 'помню',
 'кленовые',
 'аллеи,',
 'тонкий',
 'аромат',
 'опавшей',
 'листвы',
 '--',
 'запах',
 'антоновских',
 'яблок,',
 'запах',
 'меда',
 'осенней',
 'свежести.',
 'Воздух',
 'чист,',
 'точно',
 'нет,',
 'всему',
 'саду',
 'раздаются',
 'голоса',
 'скрип',
 'телег.',
 'Это',
 'тарха